In [2]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import datetime
import boto3
import botocore
import pprint
import psycopg2
import configparser

import sql_queries


#### Iniciación de Variables

In [3]:
cantidad_clientes = np.random.randint(500, 1000) #cantidad de clientes a crear
rdsIdentifier = 'banco-db-v1' #nombre de la instancia
fake = Faker() #inicialización para creación de data random

#### Cargamos archivo de configuraciones

In [4]:
config = configparser.ConfigParser()
config.read('escec.cfg')

['escec.cfg']

### Creamos Instancia de RDS 

In [5]:
aws_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                     aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                     region_name='us-east-1')

#### Verificamos Instancias de RDS disponibles

In [6]:
rdsInstanceIds = []

response = aws_conn.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['banco-db', 'banco-db-v1']


#### Creación de Servicio RDS

In [7]:
try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="postgres",
            MasterUsername=config.get('RDS', 'DB_USER'),
            MasterUserPassword=config.get('RDS', 'DB_PASSWORD'),
            Port=int(config.get('RDS', 'DB_PORT')),
            VpcSecurityGroupIds=["sg-0f6ce64d4649109a1", "sg-0f6ce64d4649109a1"],
            PubliclyAccessible=True
        )
    pprint.pprint (response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

La Instancia de Base de Datos ya Existe.


##### Recordemos Esperar unos minutos para consultar la informaicón de la instancia.

##### Obtenemos URL del Host

In [8]:
try:
     instances = aws_conn.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

banco-db-v1.cnelrr1vhuaq.us-east-1.rds.amazonaws.com


In [9]:
import sql_queries
print(sql_queries.DDL_QUERY)


CREATE TABLE IF NOT EXISTS tipo_transacciones(
    id_tipo_transac INT PRIMARY KEY,
    tipo_transaccion VARCHAR(15) UNIQUE
);

CREATE TABLE IF NOT EXISTS divisas(
    id_divisa INT PRIMARY KEY,
    nombre_divisa VARCHAR(15) UNIQUE,
    simbolo VARCHAR(2) UNIQUE
);

CREATE TABLE IF NOT EXISTS tipo_documentos(
    id_tipo_documento INT PRIMARY KEY,
    nombre_tipo_documento VARCHAR(15) UNIQUE
);

CREATE TABLE IF NOT EXISTS documentos(
    id_documento INT PRIMARY KEY,
    nombre_documento VARCHAR(50),
    uri_documento VARCHAR(100),
    fecha_hora_emision TIMESTAMP,
    tipo_documento INT,

    CONSTRAINT fk_tipo_documento
        FOREIGN KEY (tipo_documento)
            REFERENCES tipo_documentos(id_tipo_documento)

);

CREATE TABLE IF NOT EXISTS generos(
    id_genero INT PRIMARY KEY,
    genero VARCHAR(10) UNIQUE
);

CREATE TABLE IF NOT EXISTS sectores(
    id_sector INT PRIMARY KEY,
    nombre VARCHAR(50) UNIQUE,
    latitud DOUBLE PRECISION,
    longitud DOUBLE PRECISION
);

CREAT

##### Conexión a Base de Datos desde Python

In [10]:
import sql_queries

try:
    db_conn = psycopg2.connect(
        database=config.get('RDS', 'DB_NAME'), 
        user=config.get('RDS', 'DB_USER'),
        password=config.get('RDS', 'DB_PASSWORD'), 
        host=RDS_HOST,
        port=config.get('RDS', 'DB_PORT')
    )

    cursor = db_conn.cursor()
    cursor.execute(sql_queries.DDL_QUERY)
    db_conn.commit()
    print("Base de Datos Creada Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

Base de Datos Creada Exitosamente


##### Insertamos Datos en la Tabla de Transacciones

In [17]:
    #driver de conexión para insert desde pandas.

def insertDataToSQL(data_dict, table_name):
     postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    
     df_data = pd.DataFrame.from_records(data_dict)
     try:
          response = df_data.to_sql(table_name, postgres_driver, index=False, if_exists='append')
          print(f'Se han insertado {response} nuevos registros.' )
     except Exception as ex:
          print(ex)

In [34]:
data_tipo_transaciones = [
     {'id_tipo_transac': 85095, 'tipo_transaccion': 'Depóito'}, 
     {'id_tipo_transac': 85098, 'tipo_transaccion': 'Retiro'},
     {'id_tipo_transac': 85194, 'tipo_transaccion': 'Transferencia'},
     {'id_tipo_transac': 85133, 'tipo_transaccion': 'Pago Prestamo'}
]

#insertamos data en tabla tipo_transaccione 
insertDataToSQL(data_tipo_transaciones, 'tipo_transacciones')

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "tipo_transacciones_pkey"
DETAIL:  Key (id_tipo_transac)=(85095) already exists.

[SQL: INSERT INTO tipo_transacciones (id_tipo_transac, tipo_transaccion) VALUES (%(id_tipo_transac)s, %(tipo_transaccion)s)]
[parameters: ({'id_tipo_transac': 85095, 'tipo_transaccion': 'Depóito'}, {'id_tipo_transac': 85098, 'tipo_transaccion': 'Retiro'}, {'id_tipo_transac': 85194, 'tipo_transaccion': 'Transferencia'}, {'id_tipo_transac': 85133, 'tipo_transaccion': 'Pago Prestamo'})]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


##### Insertamos Datos en la Tabla de Divisas

#### Insertamos Data para Divisas

In [19]:
data_divisas = [
     {'id_divisa': 103589, 'nombre_divisa': 'Quetzales', 'simbolo': 'Q'}, 
     {'id_divisa': 114589, 'nombre_divisa': 'Dolares', 'simbolo': '$'},
     {'id_divisa': 125487, 'nombre_divisa': 'Euros', 'simbolo': '€'}
]

insertDataToSQL(data_divisas, 'divisas')

Se han insertado 3 nuevos registros.


##### Insertamos Data para tipo_documentos

In [20]:
data_tipo_documentos = [
     {'id_tipo_documento': 103589, 'nombre_tipo_documento': 'Recibo'}, 
     {'id_tipo_documento': 114589, 'nombre_tipo_documento': 'Cheque'},
     {'id_tipo_documento': 125487, 'nombre_tipo_documento': 'Contrato 101' }
]

insertDataToSQL(data_tipo_documentos, 'tipo_documentos')


Se han insertado 3 nuevos registros.


##### Insertamos Data para Generos

In [21]:
data_generos = [
     {'id_genero': 2002, 'genero': 'Maculino'}, 
     {'id_genero': 2003, 'genero': 'Femenino'}
]

insertDataToSQL(data_generos, 'generos')

Se han insertado 2 nuevos registros.


##### Insertamos Data para Sectores

In [22]:
data_sectores = [
    {'id_sector': 58974, 'nombre': 'Centro', 'latitud': 90.5689, 'longitud': 91.5687},
    {'id_sector': 58978, 'nombre': 'Centro Sur', 'latitud': 90.5981, 'longitud': 91.56878},
    {'id_sector': 58998, 'nombre': 'Centro Norte', 'latitud': 90.59875, 'longitud': 91.59897},
    {'id_sector': 58784, 'nombre': 'Sur Este', 'latitud': 90.78785, 'longitud': 91.4578},
    {'id_sector': 78786, 'nombre': 'Norte Este', 'latitud': 90.5689, 'longitud': 91.5687},
    {'id_sector': 56565, 'nombre': 'Sur Oeste', 'latitud': 90.4412, 'longitud': 91.9897},
    {'id_sector': 58889, 'nombre': 'Norte Oeste', 'latitud': 90.78745, 'longitud': 91.78897}
]
insertDataToSQL(data_sectores, 'sectores')

Se han insertado 7 nuevos registros.


##### Insertamos Data para Direcciones

In [37]:
data_direcciones = []

for index, direccion in enumerate(range(cantidad_clientes)):
    sector_random = random.sample(data_sectores,1)
    nueva_direccion = {
          'id_direccion': index,
          'no_casa': str(np.random.randint(1, 99)) + '-' + str(np.random.randint(1, 99)),
          'avenida': np.random.randint(1, 45),
          'calle':np.random.randint(1, 50),
          'zona':np.random.randint(1, 21),
          'id_sector' : sector_random[0]['id_sector']
    }

    data_direcciones.append(nueva_direccion)

insertDataToSQL(data_direcciones, 'direcciones')

(psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "direcciones_pkey"
DETAIL:  Key (id_direccion)=(0) already exists.

[SQL: INSERT INTO direcciones (id_direccion, no_casa, avenida, calle, zona, id_sector) VALUES (%(id_direccion)s, %(no_casa)s, %(avenida)s, %(calle)s, %(zona)s, %(id_sector)s)]
[parameters: ({'id_direccion': 0, 'no_casa': '33-23', 'avenida': 32, 'calle': 14, 'zona': 12, 'id_sector': 56565}, {'id_direccion': 1, 'no_casa': '91-73', 'avenida': 34, 'calle': 1, 'zona': 12, 'id_sector': 58889}, {'id_direccion': 2, 'no_casa': '20-27', 'avenida': 37, 'calle': 33, 'zona': 6, 'id_sector': 78786}, {'id_direccion': 3, 'no_casa': '79-25', 'avenida': 19, 'calle': 43, 'zona': 12, 'id_sector': 58978}, {'id_direccion': 4, 'no_casa': '73-13', 'avenida': 29, 'calle': 41, 'zona': 15, 'id_sector': 56565}, {'id_direccion': 5, 'no_casa': '74-13', 'avenida': 22, 'calle': 28, 'zona': 11, 'id_sector': 58978}, {'id_direccion': 6, 'no_casa': '69-83', 'avenida': 24, 'c

##### Insertamos Data para Clientes

In [25]:
data_clientes = []
data_direcciones_copy = data_direcciones.copy()

for cliente in range(cantidad_clientes):
    clientProfile = fake.profile()
    direccion_random = random.sample(data_direcciones_copy, 1)
    data_direcciones_copy.remove(direccion_random[0])

    nuevo_cliente = {
        'id_cliente': random.randint(10000000, 99999999),
        'nombre': clientProfile['name'].split(' ')[0],
        'apellido': clientProfile['name'].split(' ')[1],
        'correo': clientProfile['mail'],
        'fecha_nacimiento': clientProfile['birthdate'],
        'telefono': clientProfile['ssn'],
        'id_direccion': direccion_random[0]['id_direccion'],
        'id_genero': data_generos[0]['id_genero'] if(clientProfile['sex'] == 'M') else data_generos[1]['id_genero'] 
    }

    data_clientes.append(nuevo_cliente)

insertDataToSQL(data_clientes, 'clientes')

Se han insertado 693 nuevos registros.


##### Insertamos Data de Cargos

In [26]:
data_cargos = [
    {'id_cargo': 101, 'titulo_cargo': 'Cajero - Receptor'},
    {'id_cargo': 202, 'titulo_cargo': 'Gerente de Sucursal'},
    {'id_cargo': 302, 'titulo_cargo': 'Supervisor de Sector'}
]
insertDataToSQL(data_cargos, 'cargos')

Se han insertado 3 nuevos registros.


##### Insertamos Data de Colaboradores

In [27]:
cantidad_colaboradores = np.random.randint(50, 150)
data_colaboradores = []

for colab in range(cantidad_colaboradores):
    colaboradorProfile = fake.profile()
    nuevo_colaborador = {
        'id_colaborador': random.randint(10000, 99999),
        'nombre': colaboradorProfile['name'].split(' ')[0],
        'apellido': colaboradorProfile['name'].split(' ')[1], 
        'id_cargo': random.choices(data_cargos, weights=(0.6, 0.3, 0.1),  k=1)[0]['id_cargo']
    }

    data_colaboradores.append(nuevo_colaborador)
insertDataToSQL(data_colaboradores, 'colaboradores')

Se han insertado 66 nuevos registros.


##### Insertamos Data de Sucursales

In [28]:
managers = [colaborator for colaborator in data_colaboradores if colaborator['id_cargo'] == 202]
cantidad_sucursales =  len(managers)
managers_copy = managers.copy()
data_sucursales = []

for sucursal in range(cantidad_sucursales):
    #seleccionamos gerente para sucursal
    manager = random.sample(managers_copy, 1)[0]
    managers_copy.remove(manager)

    #seleccionamos sector.
    sector_random = random.sample(data_sectores,1)[0]['id_sector']
    
    #nombre de la sucursal
    sucursal_profile = fake.profile()
    nombre_sucursal = sucursal_profile['residence'].split('\n')[0]
    nombre_sucursal = ''.join([i for i in nombre_sucursal if not i.isdigit()]).strip()

    nueva_sucursal = {
        'id_sucursal': np.random.randint(180000, 190000),
        'nombre': nombre_sucursal,
        'id_sector': sector_random,
        'id_gerente': manager['id_colaborador']
    }

    data_sucursales.append(nueva_sucursal)

insertDataToSQL(data_sucursales, 'sucursales')

Se han insertado 20 nuevos registros.


##### Insertamos Datos de tipo_cuentas

In [29]:
data_tipoCuentas = [
    {'id_tipo_cuenta': 90025, 'tipo_cuenta': 'Monetaria Quetzales'},
    {'id_tipo_cuenta': 90026, 'tipo_cuenta': 'Monetaria Dolares'},
    {'id_tipo_cuenta': 90036, 'tipo_cuenta': 'Ahorro Quetzales'},
    {'id_tipo_cuenta': 90078, 'tipo_cuenta': 'Ahorro Dolares'},
    {'id_tipo_cuenta': 90044, 'tipo_cuenta': 'Cheques Quetzales'},
    {'id_tipo_cuenta': 90019, 'tipo_cuenta': 'Cheques Dolares'}
]
insertDataToSQL(data_tipoCuentas, 'tipos_cuentas')

Se han insertado 6 nuevos registros.


##### Insertamos Data para Cuentas

In [30]:
data_cuentas = []
df_data_clientes = pd.DataFrame.from_records(data_clientes)

for idnex, df_row in df_data_clientes.iterrows():
     nombre_cuenta = df_row['nombre'] + ' ' + df_row['apellido']
     nueva_cuenta = {
          'id_cuenta' : np.random.randint(1000000, 9999999),
          'nombre_cuenta' : nombre_cuenta,
          'id_tipo_cuenta': random.sample(data_tipoCuentas, 1)[0]['id_tipo_cuenta'],
          'balance': round(random.uniform(100, 1000), 2)
     }

     data_cuentas.append(nueva_cuenta)
insertDataToSQL(data_cuentas, 'cuentas')

Se han insertado 693 nuevos registros.


##### Insertamos Data de Transacciones

In [36]:
cantidad_transacciones = np.random.randint(1000, 20000)
data_transacciones = []

for transac in range(cantidad_transacciones):
    nueva_transaccion = {
            'id_transaccion': transac,
            'fecha_hora': fake.date_time_this_year(),
            'monto': round(random.uniform(1000, 10000), 2),
            'id_tipo_transaccion': random.sample(data_tipo_transaciones, 1)[0]['id_tipo_transac'],
            'id_divisa':  random.sample(data_divisas, 1)[0]['id_divisa'],
            'id_cliente': random.sample(data_clientes, 1)[0]['id_cliente'],
            'id_sucursal': random.sample(data_sucursales, 1)[0]['id_sucursal'],
            'id_colaborador': random.sample(data_colaboradores, 1)[0]['id_colaborador'],
            'id_documento_respaldo': random.sample(data_tipo_documentos, 1)[0]['id_tipo_documento'],
            'id_cuenta_origen' : random.sample(data_cuentas, 1)[0]['id_cuenta'],
            'id_cuenta_destino':random.sample(data_cuentas, 1)[0]['id_cuenta']
        }
    
    data_transacciones.append(nueva_transaccion)
    
insertDataToSQL(data_transacciones, 'transacciones')

Se han insertado 488 nuevos registros.
